In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))

In [ ]:
from katacv.utils.related_pkgs.jax_flax_optax_orbax import *
from katacv.utils.related_pkgs.utility import *
from katacv.G_VAE.parser import get_args_and_writer
args = get_args_and_writer(no_writer=True, input_args=[], model_name='G-VAE', dataset_name='cifar10')
args.path_cp = Path("/home/wty/Coding/GitHub/KataCV/logs/G-VAE-checkpoints")
args.path_dataset = Path("/home/wty/Coding/datasets/cifar10")

In [ ]:
from katacv.G_VAE.model import get_g_vae_model_state
state = get_g_vae_model_state(args)
args.load_id = 5
from katacv.utils import load_weights
state = load_weights(state, args)

In [ ]:
@jax.jit
def predict(state, x):
  return state.apply_fn(
    {'params': state.params, 'batch_stats': state.batch_stats},
    x, train=False
  )

In [ ]:
disrib, image, logits = predict(state, jnp.empty(args.input_shape))
print(disrib[0].shape, image.shape, logits.shape)

In [ ]:
from katacv.utils.mini_data.build_dataset import DatasetBuilder
from katacv.utils.mini_data.cifar10 import load_cifar10
data = load_cifar10(args.path_dataset)
ds_builder = DatasetBuilder(data, args)
ds, ds_size = ds_builder.get_dataset(subset='val', shuffle=False, use_aug=False)
from katacv.utils.mini_data.label2readable import label2readable
label2readable = label2readable['cifar10']

In [ ]:
import matplotlib.pyplot as plt
for x, y in ds:
    x, y = x.numpy(), y.numpy()
    r, c = 5, 8
    fig, axs = plt.subplots(r, c)
    for i in range(r):
        for j in range(c):
            ax = axs[i,j]
            idx = i*c+j
            ax.imshow(x[idx])
            ax.set_title(label2readable[y[idx]])
            ax.set_axis_off()
    plt.tight_layout()
    plt.show()
    break

In [ ]:
from tqdm import tqdm
import numpy as np
bar = tqdm(ds, total=ds_size)
acc = 0
for i, (x, y) in enumerate(bar):
    x, y = x.numpy(), y.numpy()
    _, _, pred_y = jax.device_get(predict(state, x))
    pred_y = np.argmax(pred_y, -1)
    # print((pred_y == y).mean())
    acc += ((pred_y == y).mean() - acc) / (i + 1)
    bar.set_description(f"Acc: {acc:.2f}")
    # break

In [ ]:
ds_train, ds_train_size = ds_builder.get_dataset(subset='train', shuffle=False, use_aug=False)
bar = tqdm(ds_train, total=ds_train_size)
acc = 0
for i, (x, y) in enumerate(bar):
    x, y = x.numpy(), y.numpy()
    _, _, pred_y = jax.device_get(predict(state, x))
    pred_y = np.argmax(pred_y, -1)
    # print((pred_y == y).mean())
    acc += ((pred_y == y).mean() - acc) / (i + 1)
    bar.set_description(f"Acc: {acc:.2f}")
    # break

In [ ]:
acc